<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np 
import pandas as pd 
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif
import sklearn.metrics as mt

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

def items_in_list(l1, l2):
    """
    Хотим найти элементы, которые есть в первом,
    но которых нету во втором листе
    :param l1: основной лист
    :param l2: второй лист
    :return: элементы, которых нету во втором листе, но которые есть в первом
    """
    u_elements = []
    common_elements = []
    for i in l1:
        if i not in l2:
            u_elements.append(i)
        else:
            common_elements.append(i)


    return u_elements, common_elements

def get_obj_cols(frame):
    return frame.select_dtypes("object").columns

def lenn(cell):
    try:
        le = len(cell.split())
        return le
    except:
        return 0

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
df_train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
df_test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
df_train.head(5)

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_train.model.value_counts()
df_train = df_train.rename(columns = {"model": "model_name"}) # переименовываем одинаковые колонки с разными названиями



In [ ]:
# адаптируем значение времени
df_test['parsing_unixtime'] = pd.to_datetime(df_test['parsing_unixtime'],unit='s').dt.date.astype('str')


## Data Preprocessing

In [ ]:
#избавляемся от пропусков
df_train.dropna(subset=['productionDate','mileage'], inplace=True)
df_train.dropna(subset=['price'], inplace=True)
df_train.dropna(subset=['bodyType', 'enginePower', 'modelDate', 'numberOfDoors', 'vehicleTransmission', 'Привод'],inplace=True)

In [ ]:
y = df_train['price']

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и валидация в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_train.reset_index(inplace=True,drop=True)

df_test["price"] = 1 # для корректной работы отмечаем цену 1 
df_test['sample'] = 0  # помечаем где у нас валидация

#Целевая переменная в обучающем датасете
y = df_train['price']

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


In [ ]:
data

In [ ]:
# дропаем признаки, которые не нужны
data.drop(columns=['car_url','complectation_dict','equipment_dict','image','model_info','name','priceCurrency','sell_id','super_gen','vehicleConfiguration',\
                   'vendor','Комплектация','hidden','Владение','Состояние','Таможня','engineDisplacement','start_date'],inplace=True)

In [ ]:
# смотрим на пропуски
nans = data.isna().sum()
print(nans)
"""
выводим колличество пропусков в колонах с пропусками
и составляем список колонок с пропусками
"""

list_of_nans = set()
for i in nans.index:
    if nans[i] > 0:
        list_of_nans.add(i)
list_of_nans = nans[list_of_nans]
print(list_of_nans)



In [ ]:
# заполняем все пропуски новой категорией
data.fillna("missed",inplace=True)

In [ ]:
data

In [ ]:
# адаптируем все названия к английскому языку
data = data.rename(columns = {'Владельцы':'owners','ПТС':'pts','Привод':'wd','Руль':'wheel'})

In [ ]:
#  сделаем списоки для каждого вида переменных
#Бинарные категориальные переменные
bin_cols = ['wheel']

#Количественные переменные
num_cols = ['enginePower', 'mileage', 'modelDate', 'numberOfDoors', 'productionDate', 'owners']

#Категориальные переменные
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'vehicleTransmission', 'pts', 'wd']

In [ ]:
#Посмотрим на значения руля
print(data['wheel'].value_counts())

In [ ]:
# приведем все названия к общему знаменателю 1 или 0
data['wheel'].replace({"LEFT":0,"Левый":0,"RIGHT":1,"Правый":1},inplace=True)

In [ ]:
#Посмотрим на значения в категориальных переменных
for i in cat_cols:
    print(i)
    print(data[i].unique())
    print(data[i].dtypes)
    print("..........................................................................")

In [ ]:
# как мы видим, часто одни и теже значения в двх датасетах называются по-разному. Нужно привести всё к одному знаменателю

color = {'040001':"чёрный", 'EE1D19': 'красный', '0000CC':'синий', 'CACECB':'серебристый',\
              '007F00':'зелёный', 'FAFBFB':'белый', '97948F':'серый', '22A0F8':'голубой',\
              '660099':'пурпурный', '200204':'коричневый', 'C49648':'бежевый',\
              'DEA522':'золотистый','4A2197':'фиолетовый', 'FFD600':'жёлтый',\
              'FF8649':'оранжевый', 'FFC0CB':'розовый'}

data['color'].replace(color,inplace=True)

trans = {'MECHANICAL': 'механическая','AUTOMATIC':'автоматическая', 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'}
data['vehicleTransmission'].replace(trans,inplace=True)

pts = {'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'}
data['pts'].replace(pts,inplace=True)

In [ ]:
# вернуться к этому моменту
#получаем топ-50
top_models = data['model_name'].value_counts().index.values.tolist()#[:150] топ-150 отключено для финальной попытки

#заменим значения моделей, не входящие в топ-50
data['model_name'] = data['model_name'].apply(lambda x: x if x in top_models else 'other')

Числовые переменные

In [ ]:
# анализируем числовые переменные
for i in num_cols:
    print(i)
    print(data[i].unique())
    print(data[i].dtypes)
    print("..........................................................................")

In [ ]:
# обработаем признак enginePower
data['enginePower'] = pd.to_numeric(data['enginePower'].replace(regex=[' N12'], value=''))

#В признаке owners заменим текстовые значения на числовые, missed = 4
data['owners'].replace({'3 или более':3.0,'1\xa0владелец':1.0,'2\xa0владельца':2,'missed':4},inplace=True)

Обработаем целевую переменную

In [ ]:
# поскольку курс рубля менялся в течении всего времени было принято решения адаптировать курс к франку
exch_rate = {'2020-10-20': 1/85.3121,'2020-10-19': 1/85.2163, '2020-10-21': 1/85.5455, '2020-10-24': 1/84.4936, '2020-10-25':1/84.4936, \
            '2020-10-26': 1/84.4936, 'missed': 1/82.8041}

data['exch_rate'] = data.parsing_unixtime.apply(lambda x: exch_rate[x])

#Удаляем не нужный признак
data.drop(columns=['parsing_unixtime'],inplace=True)

# адаптируем целевую переменную к курсу франка и логарифмируем её
data['price'] = np.log(data['price'] * data['exch_rate'])
y = y*data.query('sample == 1').copy().reset_index()['exch_rate']
y = np.log(y)

# **Feature Engeneering**

In [ ]:
data

In [ ]:
# добавим новые признаки на основе предведущих
data['mileage_per_year'] =  (data.mileage+1) / data.productionDate
data['age_year'] = 2022 - data.productionDate
data['model_age'] = 2022 - data.modelDate
data['len_of_description'] = data.description.apply(lenn)


num_cols.extend(['age_year', 'mileage_per_year', 'model_age', 'len_of_description'])

In [ ]:
data[num_cols]

In [ ]:
# добавим новые бинарные признаки
data['isnew'] = data.mileage.apply(lambda x: 1 if x == 0 else 0)
data['isdesc'] = data.len_of_description.apply(lambda x: 1 if x != 0 else 0)

# EDA

In [ ]:
# посмотрим на графики распределения
for i in num_cols:
    if i in ['numberOfDoors','owners']: continue
        
    plt.figure()
    sns.distplot(data[i][data[i] > 0], kde = False, rug=False)
    plt.title(i)
    plt.show()

In [ ]:
# применим логарифм к числовым переменным и посмотрим на графики
for i in num_cols:
    if i in ['numberOfDoors','owners']: continue
        
    data[i] = data[i].apply(lambda x: np.log(abs(x)+1))
    plt.figure()
    sns.distplot(data[i][data[i] > 0], kde = False, rug=False)
    plt.title(i)
    plt.show()

In [ ]:
data[num_cols]

In [ ]:
#построим матрицу корреляций
plt.figure(figsize=(18, 8));
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1 , annot=True, cmap='vlag')

get_dummines

In [ ]:
# Построим графики категориальных переменных
for i in cat_cols:
    plt.figure(figsize=(18, 8));
    sns.boxplot(x=i,y='price', data=data.query("sample == 1"))

Кодируем категориальные признаки

In [ ]:
# были опробованны разные виды кодирования, но этот показал лучшие результаты
for colum in cat_cols:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.head(2)

In [ ]:
#Разделяем данные 
Y = data.query('sample == 1').price
X = data.query('sample == 1').drop(['sample','exch_rate', 'price', 'description'], axis=1)

X_sub = data.query('sample == 0').drop(['sample','exch_rate','price','description'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Создание моделей

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

#Предсказываем значения
y_predict = lin_reg.predict(X_test)

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(y_predict)))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

После построения каждой модели я буду мотреть на MAPE и строить график 15 самых важных переменных для датасета

In [ ]:
"""model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')"""

# Random Forrest model

In [ ]:
"""rf_reg = RandomForestRegressor(random_state=RANDOM_SEED)
rf_reg.fit(X_train, y_train)
# оцениваем точность
predict = rf_reg.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(rf_reg.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')"""

# ExtraTreesRegressor

In [ ]:
"""etr_reg = ExtraTreesRegressor(random_state=RANDOM_SEED)
etr_reg.fit(X_train, y_train)
# оцениваем точность
predict = etr_reg.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(etr_reg.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')"""

# XGBoost

In [ ]:
"""xgb_reg = XGBRegressor()
xgb_reg.fit(X_train,y_train)
# оцениваем точность
predict = xgb_reg.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(xgb_reg.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')"""

# Meta model
финальная модель, которая показала себя лучше всего

In [ ]:
# эксперементы и построение различных моделей привели меня к тому, что лучше всего работает именно такая мета модель с тремя моделями
estimators = [('CatBoost',CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )), ('ETR', ExtraTreesRegressor(random_state=RANDOM_SEED)), ('RF', RandomForestRegressor(random_state=RANDOM_SEED))]
final_estimator = LinearRegression()
m_reg1 = StackingRegressor(estimators=estimators, final_estimator=final_estimator, n_jobs=2, cv=4)
m_reg1.fit(X_train,y_train)
predict1 = m_reg1.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(predict1)))*100:0.2f}%")




In [ ]:
"""estimators = [('CatBoost',CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )), ('ETR', ExtraTreesRegressor(random_state=RANDOM_SEED)), ('RF', RandomForestRegressor(random_state=RANDOM_SEED))]

final_estimator = RandomForestRegressor(random_state=RANDOM_SEED)
m_reg2 = StackingRegressor(estimators=estimators, final_estimator=final_estimator, n_jobs=2, cv=4)
m_reg2.fit(X_train,y_train)
predict2 = m_reg2.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(predict2)))*100:0.2f}%")"""

In [ ]:
"""estimators = [('ETR', ExtraTreesRegressor(random_state=RANDOM_SEED)), ('RF', RandomForestRegressor(random_state=RANDOM_SEED))]

final_estimator = LinearRegression()
m_reg3 = StackingRegressor(estimators=estimators, final_estimator=final_estimator, n_jobs=2, cv=4)
m_reg3.fit(X_train,y_train)
predict3 = m_reg3.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(predict3)))*100:0.2f}%")



"""

In [ ]:
"""estimators = [('CatBoost',CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )), ('ETR', ExtraTreesRegressor(random_state=RANDOM_SEED)), ('RF', RandomForestRegressor(random_state=RANDOM_SEED)),( 'XGB',XGBRegressor())]
final_estimator = LinearRegression()
m_reg1 = StackingRegressor(estimators=estimators, final_estimator=final_estimator, n_jobs=2, cv=4)
m_reg1.fit(X_train,y_train)
predict1 = m_reg1.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(predict1)))*100:0.2f}%")"""




# Submission

In [ ]:

predict_submission = np.exp(m_reg1.predict(X_sub))
predict_submission = predict_submission / data.query('sample == 0')['exch_rate']
print(predict_submission)

sample_submission['price'] = predict_submission
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head(10)


# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing